<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Final-Project-Check-in" data-toc-modified-id="Final-Project-Check-in-1">Final Project</a></span></li><li><span><a href="#Group-Name" data-toc-modified-id="Group-Name-2">Group Name</a></span></li><li><span><a href="#Student-Names" data-toc-modified-id="Student-Names-3">Student Names</a></span></li><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-4">Load Data</a></span></li><li><span><a href="#Fit-scikit-learn-model" data-toc-modified-id="Fit-scikit-learn-model-5">Fit scikit-learn model</a></span></li><li><span><a href="#Evaluation-Metric" data-toc-modified-id="Evaluation-Metric-6">Evaluation Metric</a></span></li></ul></div>

Final Project
------

Group Name: SAL
-----

Link to GitHub Repository: https://github.com/swbuchanan13/msds699_sal

Student Names
----

1. Lisa Chua
2. Alaa A Latif
3. Shane Buchanan 

In [53]:
reset -fs

## Import Packages

In [54]:
import pandas as pd
import numpy as np
from rfpimp import *
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVC
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import *
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn import compose
from sklearn.experimental import enable_iterative_imputer
from sklearn import impute
from sklearn.model_selection import KFold
from imblearn.over_sampling import SMOTE, SMOTENC
from sklearn.metrics import make_scorer
from sklearn.utils.multiclass import unique_labels

Load Data
-----

In [55]:
# Load data into Pandas DataFrame
data_url = "https://raw.githubusercontent.com/swbuchanan13/msds699_sal/master/bank-additional-full.csv"
pd.set_option('display.max_columns', None)
df = pd.read_csv(data_url, error_bad_lines=False, delimiter=';')

In [56]:
# Show Sample of data
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,261,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,149,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,226,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,151,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,307,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


In [57]:
# Show Data Type for each Variable
df.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object

In [58]:
# Display index and name of each categorical feature
t = (df.dtypes == 'object')
trues = [i for i, x in enumerate(t) if x][:-1]
for i in trues:
    print(f'{i}: {df.columns[i]}')

1: job
2: marital
3: education
4: default
5: housing
6: loan
7: contact
8: month
9: day_of_week
14: poutcome


In [59]:
# Features
X = df.drop('y', axis=1)

In [60]:
len(X.columns)

20

In [61]:
# Labels
y = df['y']

## Perform Train-Test Split

We perform a customized train-test split because the dataset is highly imbalanced. 

In [62]:
pos_samples = df[df['y']=='yes'].shape[0]
neg_samples = df[df['y']=='no'].shape[0]
print(f"Positively labeled samples: {pos_samples}")
print(f"Negatively labeled samples: {neg_samples}")

Positively labeled samples: 4640
Negatively labeled samples: 36548


In [63]:
# separate positive and negative samples
df_yes = df[df['y']=='yes']
df_no = df[df['y']=='no']
# perform train-test split on each
df_train_yes, df_test_yes = train_test_split(df_yes, test_size=0.2)
df_train_no, df_test_no = train_test_split(df_no, test_size=0.2)
# concatenate back to create a training and test set
df_train = pd.concat([df_train_yes, df_train_no], axis=0)
df_test = pd.concat([df_test_yes, df_test_no], axis=0)
# separate features from 
X_train, y_train = df_train.drop('y', axis=1), df_train['y']
X_test, y_test = df_test.drop('y', axis=1), df_test['y']

### Data Upsampling

In [64]:
def up_sample(X, y):
    rows_idxs_no = y == 'no'
    rows_idxs_yes = y == 'yes'
    df_yes = pd.concat((X.loc[rows_idxs_yes], y.loc[rows_idxs_yes]), axis = 1)
    df_no = pd.concat((X.loc[rows_idxs_no], y.loc[rows_idxs_no]), axis = 1)
    df_yes_upsampled = df_yes.sample(int(len(df_yes)*5), replace=True)
    df_upsampled = pd.concat([df_no, df_yes_upsampled], axis=0)
    return df_upsampled.drop('y', axis=1), df_upsampled['y']

In [65]:
smt = SMOTENC(trues)
X_train_up, y_train_up = smt.fit_sample(X_train, y_train)

In [66]:
X_train_up = pd.DataFrame(X_train_up, columns=X.columns)
y_train_up = pd.Series(y_train_up)

# Modeling

In [67]:
def make_pipeline(classifier=None):
    "Create a single pipeline that processes the data and then fits the classifier." 
    
    # YOUR CODE HERE
    numeric_features     = ['age', 'campaign', 'cons.price.idx', 'pdays', 'previous', 'emp.var.rate', 'cons.conf.idx',
                            'euribor3m', 'nr.employed']
    categorical_features = ['job', 'education', 'default', 'housing',
                            'loan', 'campaign', 'poutcome', 'marital', 'contact', 'month', 'day_of_week']
    
    categorical_transformer = Pipeline(steps=[
    ('impute', impute.SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))])
    
    numeric_transformer = Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='median')),
    ('scaler', preprocessing.StandardScaler())])
    
    
    
    preprocessor = compose.ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])
    
    pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                       ('classifier', classifier)])
    return pipeline

Baseline Model
----

In [68]:
# Initialize dtree and fit.
base_clf = DecisionTreeClassifier()
base_pipeline = make_pipeline(base_clf)

In [69]:
# Train Estimator
base_pipeline.fit(X_train, y_train);

In [70]:
# Predict on Test set
y_preds = base_pipeline.predict(X_test)

In [71]:
# Evaluate 
# Use accuracy score as a basic evaluation metric
accuracy_score(y_test, y_preds)

0.833818888079631

In [72]:
# Evaluate 
# Use F1 score as metric since we have an unbalanced data set.
f1_score(y_test, y_preds, average='weighted')

0.8383453110419161

## Model Selection

In [73]:
def do_model_selection():
    pipelines = {}
    models = {}
    models['logistic'] = LogisticRegression(solver='sag')
    models['random forest'] = RandomForestClassifier()
    models['GBM'] = GradientBoostingClassifier()
    models['SVM'] = SVC()
    print('Model: \t  \t Accuracy \t F1 Score')
    for name in models:
        clf = models[name]
        clf_pipeline = make_pipeline(clf)
        clf_pipeline.fit(X_train, y_train);
        y_preds = clf_pipeline.predict(X_test)
        acc = accuracy_score(y_test, y_preds)
        f1 = f1_score(y_test, y_preds, average='weighted')
        print(f'{name}: \t  \t {acc:.2f} \t {f1:.2f}')
        pipelines[name] = clf_pipeline
    return pipelines

In [ ]:
pipelines = do_model_selection()

Model: 	  	 Accuracy 	 F1 Score


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


logistic: 	  	 0.90 	 0.87


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


random forest: 	  	 0.89 	 0.87
GBM: 	  	 0.90 	 0.88


/Users/alaa/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


## Model Optimization

### Hyperparameter Tuning

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 500, num = 5)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4, 8]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pprint(random_grid)

In [ ]:
# create a scoring function to pass to the Randomized Search
f1_scorer = make_scorer(f1_score, average='weighted')
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestClassifier()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, 
                               n_iter = 10, cv = 5, verbose=2, 
                               scoring=f1_scorer, n_jobs = -1)

In [ ]:
rf_pipeline = make_pipeline(rf_random)
rf_pipeline.fit(X_train, y_train);

In [ ]:
tuned_clf = rf_pipeline['classifier']
tuned_clf.best_params_

In [ ]:
best_clf = RandomForestClassifier(n_estimators = 500, min_samples_split = 2, 
                        min_samples_leaf = 2, max_features = 'sqrt',
                        bootstrap = True)
best_pipeline = make_pipeline(best_clf)
best_pipeline.fit(X_train, y_train);
y_preds = best_pipeline.predict(X_test)
print(f"F1 Test Score: {f1_score(y_test, y_preds, average='weighted'):.2f}")
print(f"Test Accuracy: {accuracy_score(y_test, y_preds):.2f}")

## Feature Importance
### KEY TAKEAWAYS

1. <b>Intro:</b> Since our domain is business intelligence for improving tele-marketing strategy, we will focus heavily on feature importance to understand which factors play a bigger role in making the sale. 


2. <b>Overview of methods:</b> We tried three different feature importance methods: drop column, permutation and RFPimp's importance algorithm (which basically implements permutation?). We can see from the heat map that alot of the features are correlated/codependent. Therefore we decided to use the RFPimp implementation as it allows us to consider multiple features together. It avoids showing low importance for codependent features as with drop column, or having codepedent features share importance scores as with permutation.


3. <b>Interpret results:</b> The RFPimp results show <b>nr.employed, cons.conf.indx, euribor3m, month_oct</b> as the most important features. We notice that the top three features are all economic indicators. This finding may have business implications, for example:

    - Telemarketing might just not be effective in bad economic conditions and the bank should try different marketing techniques.
    - The bank should collect more economic indicators as they seem to be important factors in whether a sale is made.


4. <b>Next steps:</b> Our pipelines current model using all features, our next steps are to do more feature selection based on the results of these feature importance algorithms and to rerun our models to see if we can improve f1 score. 

In [ ]:
# Features data, drop y and Duration.
X = df.drop(['duration','y'], axis=1)
# One-hot encode categorical features.
X_dumm = pd.get_dummies(X)
# Labels data.
y = df['y']
y = y.map({'no': 0, 'yes': 1})

In [ ]:
# Train-test-split on data.
X_train, X_val, y_train, y_val = train_test_split(X_dumm, y, test_size=0.2)

In [ ]:
# RFPIMP
rf = RandomForestRegressor(n_estimators = 100, n_jobs=-1, oob_score=True)
rf.fit(X_train, y_train)
imp_rfpimp = importances(rf, X_val, y_val)
viz = plot_importances(imp_rfpimp)
viz.save('file.svg')
viz.view() # view viz in notebook

In [ ]:
# RFPIMP heatmap
df_train = feature_corr_matrix(X)
heatmap = plot_corr_heatmap(df, figsize=(8,8))
heatmap.view()

## Model Interpretation

### Change the class determination point

In [ ]:
probs_y = best_pipeline.predict_proba(X_test)[:,1]
y_pred_scale = np.where((probs_y > .2), 'yes', 'no')

In [ ]:
y_pred = best_pipeline.predict(X_test)

In [ ]:
plt.rcParams["figure.figsize"] = (8,8)

In [ ]:
def plot_confusion_matrix(y_true, y_pred,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    
    Citation:
    Function from https://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = unique_labels(y_true, y_pred)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [ ]:
np.set_printoptions(precision=2)


# # Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred_scale, normalize=True,
                      title='Business adjusted Normalized confusion matrix')

# # Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, normalize=True,
                      title='Normalized confusion matrix for Generic RF')


plt.show()

In [ ]:
y_test_binary = pd.Series(y_test).replace({'no': 0, 'yes': 1})
y_pred_binary = pd.Series(y_pred).replace({'no': 0, 'yes': 1})
lr_probs = best_pipeline.predict_proba(X_test)

# keep probabilities for the positive outcome only
lr_probs = lr_probs[:, 1]

In [ ]:
lr_precision, lr_recall, _ = precision_recall_curve(y_test_binary, lr_probs)
lr_f1, lr_auc = f1_score(y_test_binary, y_pred_binary, average='weighted'), auc(lr_recall, lr_precision)
# summarize scores
print('Random Forest: f1=%.2f auc=%.2f' % (round(lr_f1, 2), lr_auc))
# plot the precision-recall curves
plt.plot(lr_recall, lr_precision, marker='.', label='Random Forest')
# axis labels
plt.xlabel('Recall')
plt.ylabel('Precision')
# show the legend
plt.legend()
# show the plot
plt.show()

In [ ]:
np.set_printoptions(precision=2)


# Plot non-normalized confusion matrix
plot_confusion_matrix(y_test, y_pred,
                      title='Confusion matrix, without normalization')

# # Plot normalized confusion matrix
plot_confusion_matrix(y_test, y_pred, normalize=True,
                      title='Normalized confusion matrix')


plt.show()

# Key Takeaways:
Feature importance found that most of our variables are not predictive of conversion. When we compared our best f1 score to benchmarks and high scores on Kaggle, we hit a ceiling of 0.88, which may further indicate that there are missing features. Another limitation is that naïve and SMOTE upsampling to address our imbalanced data problem did not improve our model metrics. Some potential next steps would be to rerun models after feature selection and collect more economic features.